In [7]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision      # 数据库模块
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#device = torch.device("cuda")

torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 100           # 训练整批数据多少次, 为了节约时间, 我们只训练一次
BATCH_SIZE = 50
LR = 0.001          # 学习率
DOWNLOAD_MNIST = False  # 如果你已经下载好了mnist数据就写上 False

# Mnist 手写数字
train_data = torchvision.datasets.MNIST(
    root='./mnist/',    # 保存或者提取位置
    train=True,  # this is training data
    transform=torchvision.transforms.ToTensor(),    # 转换 PIL.Image or numpy.ndarray 成
                                                    # torch.FloatTensor (C x H x W), 训练的时候 normalize 成 [0.0, 1.0] 区间
    download=DOWNLOAD_MNIST,          # 没下载就下载, 下载了就不用再下了
)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

# 批训练 50samples, 1 channel, 28x28 (50, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# 为了节约时间, 我们测试时只测试前2000个
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.targets[:2000]

class EncodeModel(nn.Module):
    def __init__(self,judge=True):
        super(EncodeModel,self).__init__()
        self.encode=nn.Sequential(
                nn.Conv2d(1,16,kernel_size=7),#16*22*22
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),#16*11*11
                
                nn.Conv2d(16,4,kernel_size=3),#4*9*9
                nn.BatchNorm2d(4),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),#4*4*4
                
                nn.Conv2d(4,4,kernel_size=2),#4*3*3
                nn.BatchNorm2d(4),
                nn.ReLU(inplace=True),
            )
        self.decode=nn.Sequential(
                nn.ConvTranspose2d(4,1,kernel_size=5,stride=2),#1*9*9
                nn.BatchNorm2d(1),
                nn.ReLU(inplace=True),
                
                nn.ConvTranspose2d(1,1,kernel_size=7,stride=2),#1*23*23
                nn.BatchNorm2d(1),
                nn.ReLU(inplace=True),
                
                nn.ConvTranspose2d(1,1,kernel_size=6,stride=1),#1*28*28
                nn.BatchNorm2d(1),
                nn.ReLU(inplace=True),
            )
    def forward(self,x,judge):
        enOutputs=self.encode(x)
        outputs=self.decode(enOutputs)
        if judge:
            return outputs
        else:
            return enOutputs


class model_stage2(nn.Module):
    def __init__(self):
        super(model_stage2,self).__init__()

        self.layers=nn.Sequential(
            nn.Linear(36,10),
            nn.ReLU(inplace=True),
        )
    def forward(self,x):
        return self.layers(x)



# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Sequential(  # input shape (1, 28, 28)
#             nn.Conv2d(
#                 in_channels=1,      # input height
#                 out_channels=16,    # n_filters
#                 kernel_size=5,      # filter size
#                 stride=1,           # filter movement/step
#                 padding=2,      # 如果想要 con2d 出来的图片长宽没有变化, padding=(kernel_size-1)/2 当 stride=1
#             ),      # output shape (16, 28, 28)
#             nn.ReLU(),    # activation
#             nn.MaxPool2d(kernel_size=2),    # 在 2x2 空间里向下采样, output shape (16, 14, 14)
#         )
#         self.conv2 = nn.Sequential(  # input shape (16, 14, 14)
#             nn.Conv2d(16, 32, 5, 1, 2),  # output shape (32, 14, 14)
#             nn.ReLU(),  # activation
#             nn.MaxPool2d(2),  # output shape (32, 7, 7)
#         )
#         self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = x.view(x.size(0), -1)   # 展平多维的卷积图成 (batch_size, 32 * 7 * 7)
#         output = self.out(x)
#         return output


def train(traindata,model):
    #model=EncodeModel(True)
    optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler=StepLR(optimizer,step_size=5,gamma=0.8)
    criterion=nn.MSELoss()
    epochs=5  
    for epoch in range(epochs):
        for j,(trainData,i) in enumerate(traindata):
            outputs=model(trainData,True)
            optimizer.zero_grad()
            loss=criterion(outputs,trainData)
            loss.backward()
            optimizer.step()
            torch.save(model.state_dict(),'EncodeModel.pth')
        print('epoch:{} loss:{:7f}'.format(epoch,loss.item()))
        scheduler.step()

def train_stage2(traindata,model):
    optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler=StepLR(optimizer,step_size=5,gamma=0.8)
    criterion=nn.MSELoss()
    epochs=5
    for epoch in range(epochs):
        for i,(data,label) in enumerate(traindata):
            output = model(data)
            optimizer.zero_grad()
            loss=criterion(output,label)
            print(loss)
            loss.backward(retain_graph=True)
            optimizer.step()
            torch.save(model.state_dict(),'Model2.pth')

        print('epoch:{} loss:{:7f}'.format(epoch,loss.item()))
        scheduler.step()


def show_result(traindata,model):
    for j,(trainData,y_data) in enumerate(traindata):
        input = trainData
        output = model(trainData,True)
        input = (torchvision.utils.make_grid(input).permute((1,2,0))).detach().numpy()
        print(input)
        #plt.figure(1)
        #plt.imshow((torchvision.utils.make_grid(input).permute((1,2,0))).detach().numpy())
        #plt.show()        

In [3]:
model = EncodeModel()
model.load_state_dict(torch.load("EncodeModel.pth"))
model2 = model_stage2()
tensorlist=list()

for step,(traindata,label) in enumerate(train_loader):
    output = model(traindata,False)
    output=output.view(50,36)
    tensorlist.append(output)
final_tensor = torch.stack(tensorlist)
final_tensor = final_tensor.view(1200*50,36)
final_tensor = final_tensor.float()

D:\Users\DELL\Anaconda3\envs\py38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
final_label = list()
for step,(traindata,label) in enumerate(train_loader):
    for (j,i) in enumerate(label.tolist()):
        temp = np.zeros(10).tolist()
        temp[i]=1
        final_label.append(torch.from_numpy(np.array(temp)))
final_label=torch.stack(final_label)
final_label=final_label.float()
final_dataset=Data.TensorDataset(final_tensor,final_label)
final_data = Data.DataLoader(dataset=final_dataset, batch_size=50, shuffle=True)

In [6]:
model2=model2.cuda()
#model2.load_state_dict(torch.load("Model2.pth"))
train_stage2(final_data,model2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument mat1 in method wrapper_addmm)